In [30]:
import numpy as np

In [31]:
# Constant number of decimal bits use for fixed-point presentation
DEC_BITS = 5 # Increment number, increment precision. But, we have to face with
            # trade-off between accuracy and memory.

# Both weights and biases may positive or negative number
# Assigns the list with a set of floating-point number
list_nums = [-1.0, 2.3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [32]:
# signed float to unsigned fixed point (2^5 = 32 bits = 4 bytes per data)
converted_nums = []
for list_num in list_nums:
    # Converts the current element to a fixed-point presentation
    converted_num = int(np.round(list_num * (2**DEC_BITS)))
    if converted_num < 0:
        converted_num = converted_num + 2**32 # signed to unsigned
    converted_nums.append(converted_num)

In [33]:
# unsinged to string (eg: 1 to '00000001')
string_nums = []
for i in converted_nums:
    string_num = str(hex(int(i)).upper()[2:]).zfill(8) # eg: OX00000001 => 00000001
    string_nums.append(string_num)

In [34]:
"""To initialize the M10k, we use intel hex file to initialize the M10k in the IP catalog. An simple example of the hex file could be as follows:
:040001000000004AB1
:00000001FF
The first line, from left to right, : stands for a start code of a record. 04 means the record length is 4 bytes, and 0001 means the address is 1.
The following 00 means this is a data record. Next 0000004A means the data 0x000004A will be stored.
The B1 in the end is the checksum of the data record. The second line, :00000001FF stands for the end of the file."""

for k, num in enumerate(string_nums):
    string_nums[k] = '04' + str(hex(k).upper()[2:].zfill(4)) + '00' + num
    if k >= 0xffff:
        print("Too Many Data!")
    data = bytearray.fromhex(string_nums[k])
    check_sum = (0x100 - sum(data)) & 0xff
    string_nums[k] = string_nums[k] + str(hex(check_sum).upper()[2:].zfill(2))
    

In [35]:
# calculate checksum, and add end of file
# Needs creating a m10k.hex file to represent binary data in a human readabel format.
# M10K file will be interpreted by hardware systems.
with open('m10k.hex', 'w') as f:
    for line in string_nums:
        line = ':' + line
        f.write(line)
        f.write('\n')
    end = ":00000001FF"
    f.write(end)
    f.write('\n')

In [ ]:
!jupyter nbconvert --to webpdf --allow-chromium-download hex_generator.ipynb